In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from qutip import *
from scipy import *
from numpy import *
from scipy.optimize import minimize
import random
from math import log

from IPython.display import display

from sympy.interactive import printing
printing.init_printing()

from collections import defaultdict

from __future__ import division

from random import randrange
from datetime import datetime

from tempfile import TemporaryFile
arglikelihood = TemporaryFile()
argFid = TemporaryFile()

In [5]:
j = complex(0,1)

class PAULI (object) :
    
    name = ''
    op = Qobj()
    
    def __init__(self, name, PauliMatrix) :
        self.name = name
        self.op = PauliMatrix
    
def make_pauli_matrix (name, PauliMatrix) :
    Pmatrix = PAULI (name, PauliMatrix)
    return Pmatrix

 
sx = make_pauli_matrix('X', sigmax())
sy = make_pauli_matrix('Y', sigmay())
sz = make_pauli_matrix('Z', sigmaz())
I = make_pauli_matrix('I', qeye(2))

######################################################
#GATE and DIMENSION
####################################################
N = 4        # n of qubits

GateMatrix = matrix([[1,0,0,0,0,0,0,0],
                     [0,1,0,0,0,0,0,0],
                     [0,0,1,0,0,0,0,0],
                     [0,0,0,1,0,0,0,0],
                     [0,0,0,0,1,0,0,0],
                     [0,0,0,0,0,1,0,0],
                     [0,0,0,0,0,0,1,0],
                     [0,0,0,0,0,0,0,-1]]) #GATE ODD
#G = Qobj(GateMatrix, dims = [[2,2,2],[2,2,2]])
#G = toffoli()
G = fredkin()

interactions = [[0,1],[0,2],[0,3],[0,4],[1,2],[1,3],[1,4],[2,3],[2,4]]

#######################################################################
#USEFUL DEFINITIONS
#######################################################################

dimG = G.shape[0]  
CareStateDim = int(log(dimG,2))

h = N-CareStateDim

sq2 = 1/sqrt(2)

if h > 1 :
    dontCareStates = [sq2*basis(2,1) + sq2*basis(2,0)]*(h)
    dontCareIdentity = [qeye(2)]*(h)
else :
    dontCareStates = [sq2*basis(2,1) + sq2*basis(2,0)]
    dontCareIdentity = [qeye(2)]


dCS = tensor(dontCareStates)
dCI = tensor(dontCareIdentity)

###########################################################################
#FUNCTIONS
###########################################################################

def Likelihood(J, rho_0) : #Likelihood function
    
    Ak = G*rho_0
    #A = tensor(Ak*Ak.dag(), dCI)
    A = Ak*Ak.dag()
    
    rho = tensor(rho_0, dCS)
    H = HamiltonianAB(J)
    U = (-j*H).expm()
    Btemp = U*rho
    Bk = Btemp*Btemp.dag()
    B = Bk.ptrace([0,1,2])
    
    out = (A*B).tr()

    return abs(out)



def HamiltonianAB(x,N) :
    k = 0
    H = 0
    sx = sigmax()
    sy = sigmay()
    sz = sigmaz()
    for q in [sx,sy,sz]:    
        temp = 0
        OpChain = [qeye(2)]*N
        OpChain[2] = q
        OpChain[1] = q
        temp += x[k]*tensor(OpChain)
        k+=1        
        H += temp 
    for p in [0,3]:
        for q in [sx,sz]:
            temp = 0
            OpChain = [qeye(2)]*N
            OpChain[2] = q
            OpChain[p] = q
            temp += x[k]*tensor(OpChain)
            OpChain = [qeye(2)]*N
            OpChain[1] = q
            OpChain[p] = q
            temp += x[k]*tensor(OpChain)
            k += 1
            H += temp 
    for q in [sx,sz]:
        temp = 0
        OpChain = [qeye(2)]*N
        OpChain[0] = q
        OpChain[3] = q
        temp += x[k]*tensor(OpChain)
        k+=1        
        H += temp 
    for i in range(4) :
        temp = 0
        OpChain = [qeye(2)]*N
        OpChain[i] = sz
        temp += x[k]*tensor(OpChain)
        H += temp 
        k += 1
    
    temp = 0
    OpChain = [qeye(2)]*N
    OpChain[3] = sx
    temp += x[k]*tensor(OpChain)#last one

    H += temp 

    
    return H



In [6]:
###########################################
##Functions to implement the Fidelity
###########################################

def getGate (G): #extract non zero elements of the gate and save them in s
    
    s = []

    rows = G.shape[0]
    colums = G.shape[1]
    
    for i in range(rows):
        for j in range(colums):
            if G[i][0][j] != 0 :
                s.append([i,j])
    return s

def bin(a):  #binary representation of a number a: useful to write the computational basis 
    s=''                                          #inside the Fidelity
    t={'0':'000','1':'001','2':'010','3':'011',
       '4':'100','5':'101','6':'110','7':'111'}
    for c in oct(a)[1:]:
            s+=t[c]
    return s

def getBasis (a) : #get the basis states according to the binary of a: 10 -> |10>
    
    if a == 0:
          B = [basis(2,0)]*(CareStateDim)
          return tensor(B)
    
    c = bin(a)
    if dimG == 8:
        return tensor(basis(2,int(c[0])) , basis(2,int(c[1])), basis(2,int(c[2])))
    if dimG == 4:
        return tensor(basis(2,int(c[1])) , basis(2,int(c[2])))
    
def Fidelity (J):  #Fidelity function
    
    s = getGate(G)
    H = HamiltonianAB(J,N)
    Fid = 1/(dimG + 1)
    U = (-j*H).expm()
    Udag = U.dag()
    
    for x in s :
        for y in s:
            
            #definition of the basis kets and bras.             
            bra_i = getBasis(x[0]).dag()
            ket_j = getBasis(y[0])
            ket_k = getBasis(x[1])
            bra_l = getBasis(y[1]).dag()
            
            Epsilon = U*tensor(ket_k*bra_l, dCS*dCS.dag())*Udag
            Eps_ijkl = bra_i*(Epsilon.ptrace([0,1,2]))*ket_j
           
            
            Gstar_ik = G[x[0],x[1]].conjugate()
            G_jl = G[y[0],y[1]]
            
            
            fidStep = (1/(dimG*(dimG+1)))*Gstar_ik*Eps_ijkl*G_jl     
            Fid += fidStep[0][0][0]
            
    return abs(Fid)

In [9]:
def LocGradient(J) :
    
    dL = []
    delta = 0.0001
    for s in range(len(J)) :
        JdJ = [x for x in J]
        JdJ[s] += delta
        
        dL.append((Fidelity(JdJ) - Fidelity(J))/delta)
    
    return dL

def GradDesc(Jopt) :
    
    J = [x for x in Jopt]
    check = 0
    s = 1
    i = 1
    g = 0
    
    while True :   
        
        g = s
        grad = LocGradient(J)
        J = [J[k] + grad[k]/i for k in range(len(J))]
    
        s = Fidelity(J) 
        t.write(str(i)+'    '+str(s)+'\n')
        

        print(s)
        
        if abs(s-g) < 0.00005 :
            check += 1    
            if check == 20:
                break
        
        i += 1
       
    return J

t = open('toffGRAD', 'w+')
J2 = GradDesc(Jopt)
t.close()

0.919532965563
0.919804067719
0.919895738326
0.919899733508
0.919901531696
0.919902563421
0.919903219451
0.919903666139
0.919903985874
0.919904223673
0.919904405982
0.919904549238
0.919904664131

KeyboardInterrupt: 

In [6]:
########################
#SGD
######################

step = 0.0008

t = open('toffoliGRAD', 'w+')
startTime = datetime.now()

Jopt = rand(2*len(interactions)+3)


delta = 0.0001
check = 0
s = 0
time = step


for i in range(3):
    
    J = [x for x in Jopt]
    
    walk = step*500*i
    
    while True :
        
        walk += step
        time += step
        
        rho_0 = rand_ket(N = 8, dims = [[2,2,2], [1,1,1]])
        index = randrange(len(J))
        
        JdJ = [x for x in J]
        JdJ[index] += delta
        grad = (Likelihood(JdJ, rho_0) - Likelihood(J, rho_0))/delta
        J[index] = J[index] + grad/sqrt(walk)
        s = Likelihood(J, rho_0) 
        
        
        
        t.write(str(time/step)+'    '+str(s)+'\n')        
        
        
        
        if abs(s) > 0.99 :
            check += 1    
            if check == 20:
                break
         
        if time/step % 100 < 1:
            print(str(time/step)+ '   ' + str(s)+ '   ' + str(walk))
            
            
        if time/step > 1600*(i+1) :
            print('HIT')
            break
    
    Jopt = J


print(datetime.now()-startTime)
print(Jopt)
t.close()
alert.play()

TypeError: HamiltonianAB() takes exactly 2 arguments (1 given)

In [22]:
from decimal import *
getcontext().prec = 5
i = s = 0
N = 5
Jopt = J2
zz = [sz,sz]

s += 1
c = 'J_' + sz.name + sz.name
J = zeros(shape=(N,N))
for k in interactions :
    a = "{0:.3f}".format(Jopt[i])
    val = float(a)
    J[k[0],k[1]] = val
    i += 1
i = s
exec(c + '= asmatrix(J)')

In [23]:
J_ZZ

matrix([[ 0.   ,  1.196,  1.215, -0.   , -0.   ],
        [ 0.   ,  0.   , -0.382, -0.   ,  1.571],
        [ 0.   ,  0.   ,  0.   , -0.   ,  1.571],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [24]:
J_ZZ[3,4]

In [32]:
J_IZ

matrix([[ 0.   , -2.26 , -1.218,  0.978,  0.406],
        [ 0.   ,  0.   ,  0.311,  0.239,  0.863],
        [ 0.   ,  0.   ,  0.   ,  0.708,  0.429],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [33]:
J_ZI

matrix([[ 0.   ,  3.496, -1.005, -2.924,  1.558],
        [ 0.   ,  0.   ,  2.572,  6.592,  0.598],
        [ 0.   ,  0.   ,  0.   ,  1.07 ,  1.041],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])